# DPL301

In [4]:
### Install dependencies
!pip install -q transformers datasets evaluate apache_beam sentencepiece

In [5]:
from transformers import DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, concatenate_datasets
import datasets

In [6]:
# load dataset
dataset = load_dataset('glue',"cola")
# print(dataset)
dataset_train = dataset['train']
dataset_val = dataset['validation']

def preprocess(example):
    
    encoded = tokenizer.encode_plus(example['sentence'],return_tensors='pt')

    input_ids_list = []
    target_ids_list = []
    
    word_ids = encoded.word_ids()
    input_ids = encoded['input_ids'][0]
    
    i = 1
    while input_ids[i] != tokenizer.eos_token_id:
        init_i = i
        while word_ids[i] == word_ids[i+1]:
            i += 1
        input_ids_list.append(input_ids[:init_i])
        target_ids_list.append(input_ids[init_i:i+1])
        i += 1

    inputs = tokenizer.batch_decode(input_ids_list,skip_special_token=True)
    labels = tokenizer.batch_decode(target_ids_list,skip_special_token=True)
    return {'inputs':inputs,'labels':labels}

def preprocess_data(dataset):
    data = dataset.map(preprocess)
    data = data.remove_columns(['sentence','label','idx'])
    d_input = data.map(lambda x: {"input_ids": [tokenizer(i)['input_ids'] for row in x["inputs"] for i in row] }, batched=True, remove_columns=['labels','inputs'])
    d_target = data.map(lambda x: {"labels": [tokenizer(text_target=i)['input_ids'] for row in x["labels"] for i in row] }, batched=True, remove_columns=['inputs'])
    d_mask = data.map(lambda x: {"attention_mask": [tokenizer(i)['attention_mask'] for row in x["inputs"] for i in row] }, batched=True, remove_columns=['labels','inputs'])
    
    data = concatenate_datasets([d_input, d_target, d_mask], axis=1)
    return data
q = dataset_train.map(preprocess)
p = preprocess_data(dataset_train)

  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

NameError: ignored

## Allen AI

In [7]:
# no tuning
# TODO: add evaluation metrics (accuracy by exact string matching) 

tokenizer = AutoTokenizer.from_pretrained("allenai/t5-small-next-word-generator-qoogle")
model = AutoModelForSeq2SeqLM.from_pretrained("allenai/t5-small-next-word-generator-qoogle")

data_collator = DataCollatorForSeq2Seq(model=model,tokenizer=tokenizer,return_tensors='pt')

In [8]:
train_data = preprocess_data(dataset_train).shuffle()
val_data = preprocess_data(dataset_val)

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

In [9]:
# evaluation code using exact_match
import numpy as np
from evaluate import load

exact_match_metric = load("exact_match")
def metrics_compute(EvalPrediction):
  predictions, labels = EvalPrediction
  decoded_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  return exact_match_metric.compute(predictions=decoded_predictions, references=decoded_labels)
  
# results = exact_match_metric.compute(predictions=predictions, references=references)

In [ ]:
args = Seq2SeqTrainingArguments(
    f"allen_ai_t5",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    fp16=True,

    evaluation_strategy = "steps",
    per_device_eval_batch_size=32,
    eval_steps=500,
    predict_with_generate=True,
    generation_max_length=10,
    generation_num_beams=2,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=metrics_compute
)

Using cuda_amp half precision backend


In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 58444
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3654
  Number of trainable parameters = 60506624
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Exact Match
500,3.240400,2.829474,0.146551
1000,2.921400,2.704511,0.148864
1500,2.818000,2.646223,0.149680
2000,2.796500,2.610627,0.151857
2500,2.745100,2.587525,0.153490
3000,2.750900,2.574383,0.153626
3500,2.709000,2.569324,0.154579


Streaming output truncated to the last 5000 lines.
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26

TrainOutput(global_step=3654, training_loss=2.847195847150597, metrics={'train_runtime': 793.0125, 'train_samples_per_second': 147.397, 'train_steps_per_second': 4.608, 'total_flos': 610246666616832.0, 'train_loss': 2.847195847150597, 'epoch': 2.0})

In [ ]:
# generate text until the output length (which includes the context length) reaches 50
inputs = tokenizer("Hello, my dog is cute and ", return_tensors="pt").to("cuda")
generation_output = model.generate(**inputs, return_dict_in_generate=True, output_scores=True)# min lengbeam_search, max length
# output = tokenizer.decode(generation_output["sequences"], skip_special_tokens=True)
sequence =generation_output["sequences"]

output = tokenizer.batch_decode(sequence, skip_special_tokens=True)

output


Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['swollen']

## Flan-T5

In [ ]:
# tuning
# TODO: add tune code

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
#add data collator_tnm
data_collator = DataCollatorForSeq2Seq(model=model,tokenizer=tokenizer,return_tensors='pt')


loading file spiece.model from cache at /root/.cache/huggingface/hub/models--google--flan-t5-small/snapshots/9471d3bc4f85c9012776f03c4c00fdfe0d789a95/spiece.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-small/snapshots/9471d3bc4f85c9012776f03c4c00fdfe0d789a95/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-small/snapshots/9471d3bc4f85c9012776f03c4c00fdfe0d789a95/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-small/snapshots/9471d3bc4f85c9012776f03c4c00fdfe0d789a95/tokenizer_config.json


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-small/snapshots/9471d3bc4f85c9012776f03c4c00fdfe0d789a95/config.json
Model config T5Config {
  "_name_or_path": "google/flan-t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
  

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--google--flan-t5-small/snapshots/9471d3bc4f85c9012776f03c4c00fdfe0d789a95/pytorch_model.bin
Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

All model checkpoint weights were used when initializing T5ForConditionalGeneration.

All the weights of T5ForConditionalGeneration were initialized from the model checkpoint at google/flan-t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use T5ForConditionalGeneration for predictions without further training.


loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--google--flan-t5-small/snapshots/9471d3bc4f85c9012776f03c4c00fdfe0d789a95/generation_config.json
Generate config GenerationConfig {
  "_from_model_config": true,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}



In [ ]:
 # evaluation code using exact_match

# from evaluate import load
# exact_match_metric = load("exact_match")
# results = exact_match_metric.compute(predictions=predictions, references=references)

In [11]:
train_data = preprocess_data(dataset_train).shuffle()
val_data = preprocess_data(dataset_val)

In [17]:
args = Seq2SeqTrainingArguments(
    f"google_flan_t5",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    fp16=False,

    evaluation_strategy = "steps",
    per_device_eval_batch_size=32,
    eval_steps=500,
    predict_with_generate=True,
    generation_max_length=10,
    generation_num_beams=2,
)
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=metrics_compute
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [18]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 58444
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3654
  Number of trainable parameters = 60506624
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [ ]:


# generate text until the output length (which includes the context length) reaches 50
inputs = tokenizer("Hello, my dog is cute and ", return_tensors="pt").to("cuda")
generation_output = model.generate(**inputs, return_dict_in_generate=True, output_scores=True)# min lengbeam_search, max length
# output = tokenizer.decode(generation_output["sequences"], skip_special_tokens=True)
sequence =generation_output["sequences"]

output = tokenizer.batch_decode(sequence, skip_special_tokens=True)

output

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['a']

## XLnet

In [98]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [99]:
tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")

model = AutoModelForCausalLM.from_pretrained("xlnet-base-cased")

data_collator = DataCollatorForSeq2Seq(model=model,tokenizer=tokenizer,return_tensors='pt')

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlnet-base-cased/snapshots/ceaa69c7bc5e512b5007106a7ccbb7daf24b2c79/config.json
Model config XLNetConfig {
  "_name_or_path": "xlnet-base-cased",
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "model_type": "xlnet",
  "n_head": 12,
  "n_layer": 12,
  "pad_token_id": 5,
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_dropout": 0.1,
  "summary_type": "last",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sampl

In [100]:
a = tokenizer('hello this is a test',return_tensors='pt',text_target='hello this is a test')
label = tokenizer(text_target='hello this is a test',return_tensors='pt')
print(a)
print(label)
result = model(input_ids=a['input_ids'],attention_mask=a['attention_mask'],labels=label['input_ids'])
print(result.keys())

{'input_ids': tensor([[24717,    52,    27,    24,   934,     4,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[24717,    52,    27,    24,   934,     4,     3]])}
{'input_ids': tensor([[24717,    52,    27,    24,   934,     4,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}
odict_keys(['loss', 'logits', 'mems'])


In [113]:
# load dataset
dataset = load_dataset('glue',"cola")
# print(dataset)
dataset_train = dataset['train']
dataset_val = dataset['validation']

def preprocess_data(dataset):
    d = dataset.map(lambda x: tokenizer(x['sentence']), batched=True, remove_columns=['label','idx','sentence'])
    return d.map(lambda x: {'labels': x['input_ids']})
train_data = preprocess_data(dataset_train).shuffle()
val_data = preprocess_data(dataset_val)

  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

In [116]:
print(train_data['labels'])

[[346, 12412, 27, 24, 293, 269, 29482, 4546, 28, 110, 22, 210, 22, 714, 13230, 21, 333, 25, 24, 3989, 14156, 9, 4, 3], [35, 569, 491, 12434, 1421, 13366, 9, 4, 3], [147, 26, 23, 456, 4719, 9, 4, 3], [15739, 9459, 110, 24, 1160, 9, 4, 3], [35, 175, 35, 170, 216, 22, 18, 14739, 26, 23, 19, 57, 35, 125, 220, 26, 46, 210, 22, 9, 4, 3], [32, 226, 797, 302, 81, 1118, 5654, 7652, 22, 222, 493, 18, 109, 8478, 9, 4, 3], [24159, 17, 1167, 18, 23186, 9, 4, 3], [69, 9772, 645, 2123, 31, 18, 291, 33, 24, 17, 6319, 18914, 1197, 9, 4, 3], [21404, 1695, 40, 18, 2551, 9, 4, 3], [69, 53, 47, 72, 2531, 45, 341, 9, 4, 3], [147, 26, 23, 125, 456, 4719, 9, 4, 3], [35, 685, 44, 105, 21, 61, 17, 1167, 113, 82, 4, 3], [324, 1922, 36, 31, 92, 140, 1206, 24, 699, 19, 314, 26, 46, 36, 82, 4, 3], [408, 263, 172, 319, 9, 4, 3], [1597, 19559, 151, 18, 2102, 9, 4, 3], [29968, 51, 28, 993, 445, 4083, 292, 9, 4, 3], [609, 2467, 255, 27, 1597, 5694, 9, 4, 3], [200, 3859, 25, 18, 1645, 9, 4, 3], [6006, 255, 300, 522, 85,

In [121]:
from transformers import TrainingArguments, Trainer
args = TrainingArguments(
    f"xlnet_based_case",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_steps=1,
    num_train_epochs=2,
    fp16=False,

    evaluation_strategy = "steps",
    per_device_eval_batch_size=32,
    eval_steps=500,
)

trainer = Trainer(
    model,
    args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=metrics_compute
)


TypeError: ignored

In [122]:
trainer.train()

***** Running training *****
  Num examples = 8551
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 536
  Number of trainable parameters = 116750336


Step,Training Loss,Validation Loss


KeyboardInterrupt: ignored